In [19]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action="ignore")
pd.set_option('display.max_columns', None)
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import resample
from sklearn.impute import KNNImputer
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, VotingClassifier, HistGradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

In [2]:
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic' # 예시: 맑은 고딕으로 설정 (Windows 환경 시)
plt.rcParams['axes.unicode_minus'] = False

In [17]:
data = pd.read_csv("건강데이터_2022_2023_합본.csv")

In [18]:
data['LS_VEG1'].value_counts()

LS_VEG1
2.0     5685
1.0     5482
3.0     1234
5.0      121
4.0       80
6.0       13
7.0        4
9.0        4
99.0       4
8.0        2
Name: count, dtype: int64

전처리 

In [ ]:
data['BE5_1'] = data['BE5_1'].astype(str).str.strip().replace({ #1주일간 근력운동 일수
'1.0' : 0,
'2.0' : 1,
'3.0' : 2,
'4.0' : 3,
'5.0' : 4,
'6.0' : 5,
'8.0' : 0,
'9.0' : np.nan,
'nan' : np.nan
 })
data['BE5_1'] = pd.to_numeric(data['BE5_1'], errors='coerce')

In [ ]:
data['LS_VEG1'] = data['LS_VEG1'].astype(str).str.strip().replace({  #최근 1년 동안 평균 채소류(김치 및 장아찌 제외), 버섯류, 해조류 섭취 빈도
'1.0' : 1095,
'2.0' : 730,
'3.0' : 365,
'4.0' : 286,
'5.0' : 156,
'6.0' : 52,
'7.0' : 30,
'8.0' : 12,
'9.0' : 6,
'99.0': np.nan,
'nan' : np.nan
})
data['LS_VEG1'] = pd.to_numeric(data['LS_VEG1'], errors='coerce')

In [ ]:
# 일반 담배 하루 평균 흡연량 처리
data['BS3_2'] = data['BS3_2'].replace(888, 0)

# 전자 담배 하루 평균 흡연량 처리
data['BS12_47_1'] = data['BS12_47_1'].replace({888: 0, 999: np.nan}) 

# 합치기 전 숫자형으로 변환
data['BS3_2'] = pd.to_numeric(data['BS3_2'], errors='coerce')
data['BS12_47_1'] = pd.to_numeric(data['BS12_47_1'], errors='coerce')

# 두 변수 합산 (NaN은 무시하고 합함)
data['tobacco'] = data[['BS3_2', 'BS12_47_1']].sum(axis=1, skipna=True)


In [ ]:
data['BD1_11'] = data['BD1_11'].astype(str).str.strip().replace({ #1년간 음주빈도
'1.0' : 0,
'2.0' : 6,
'3.0' : 12,
'4.0' : 42,
'5.0' : 130,
'6.0' : 286,
'8.0' : 0,
'9.0' : np.nan,
'nan' : np.nan
})
data['BD1_11'] = pd.to_numeric(df['BD1_11'], errors='coerce')

In [ ]:
import joblib

# 결측치 비율 25% 이상인 컬럼 제거
missing_ratio = data.isnull().mean()
columns_to_drop = [col for col in ['HE_Ualb', 'HE_HTG'] if missing_ratio[col] > 0.25]
data = data.drop(columns=columns_to_drop, errors='ignore')
print(f"Removed columns: {columns_to_drop}")


# HE_DM_HbA1c
if 'HE_DM_HbA1c' in data.columns:
    data['HE_DM_HbA1c'] = data['HE_DM_HbA1c'].replace([9, 99, 999, 888, 9.0, 99.0, 999.0, 888.0], np.nan)
    if 'DE1_dg' in data.columns:
        print(f"DE1_dg missing ratio: {data['DE1_dg'].isnull().mean()}")
        data['HE_DM_HbA1c'] = data['HE_DM_HbA1c'].fillna(data['DE1_dg'].map({0: 1, 1: 3}))
    # 남은 결측치는 클래스 비율 기반 랜덤 샘플링
    if data['HE_DM_HbA1c'].isnull().any():
        class_probs = data['HE_DM_HbA1c'].value_counts(normalize=True)
        data['HE_DM_HbA1c'] = data['HE_DM_HbA1c'].apply(
            lambda x: np.random.choice(class_probs.index, p=class_probs.values) if pd.isna(x) else x
        )
    print(f"HE_DM_HbA1c NaN count after: {data['HE_DM_HbA1c'].isnull().sum()}")

# HE_obe
if 'HE_obe' in data.columns:
    data['HE_obe'] = data['HE_obe'].replace([9, 99, 999, 888, 9.0, 99.0, 999.0, 888.0], np.nan)
    if 'HE_BMI' in data.columns:
        # BMI가 KNN 대체된 경우 표시
        data['HE_BMI_imputed'] = data['HE_BMI'].isnull().astype(int)
        data['HE_obe'] = data.apply(
            lambda row: row['HE_obe'] if pd.notna(row['HE_obe']) else (
                1 if row['HE_BMI'] < 18.5 else
                2 if row['HE_BMI'] < 25 else
                3 if row['HE_BMI'] < 30 else
                4 if row['HE_BMI'] < 35 else
                5 if row['HE_BMI'] < 40 else
                6
            ),
            axis=1
        )
    # 남은 결측치는 클래스 비율 기반 랜덤 샘플링
    if data['HE_obe'].isnull().any():
        class_probs = data['HE_obe'].value_counts(normalize=True)
        data['HE_obe'] = data['HE_obe'].apply(
            lambda x: np.random.choice(class_probs.index, p=class_probs.values) if pd.isna(x) else x
        )
    print(f"HE_obe NaN count after: {data['HE_obe'].isnull().sum()}")

# HE_HP
if 'HE_HP' in data.columns:
    data['HE_HP'] = data['HE_HP'].replace([9, 99, 999, 888, 9.0, 99.0, 999.0, 888.0], np.nan)
    if 'DI1_dg' in data.columns:
        print(f"DI1_dg missing ratio: {data['DI1_dg'].isnull().mean()}")
        data['HE_HP'] = data['HE_HP'].fillna(data['DI1_dg'].map({0: 1, 1: 4}))
    # 남은 결측치는 클래스 비율 기반 랜덤 샘플링
    if data['HE_HP'].isnull().any():
        class_probs = data['HE_HP'].value_counts(normalize=True)
        data['HE_HP'] = data['HE_HP'].apply(
            lambda x: np.random.choice(class_probs.index, p=class_probs.values) if pd.isna(x) else x
        )
    print(f"HE_HP NaN count after: {data['HE_HP'].isnull().sum()}")

#결측치 비율 낮은 변수 처리
    low_missing_columns = [
    'incm', 'ho_incm', 'edu', 'occp', 'HE_fh', 'D_1_1', 'DI1_dg', 'DI1_ag', 'DI1_pr', 'DI1_pt', 'DI1_2',
    'DI2_dg', 'DI2_ag', 'DI2_pr', 'DI2_pt', 'DI2_2', 'DE1_dg', 'DE1_ag', 'DE1_pr', 'DE1_pt', 'DE1_3',
    'DE1_31', 'DE1_32', 'DE1_33', 'DE1_34', 'DE1_4', 'DN1_dg', 'DN1_ag', 'BH1', 'BO1', 'BO1_1', 'BO2_1',
    'BD1', 'BD1_11', 'BD2_1', 'BP16_1', 'BP16_2', 'BP1', 'BP5', 'BS1_1', 'BS3_1', 'BS3_2', 'BS12_37',
    'BS12_47', 'BS12_47_1', 'BS12_1', 'BS12_2', 'BE3_31', 'BE3_32', 'BE3_33', 'BE5_1', 'BE9', 'L_BR_FQ',
    'L_LN_FQ', 'L_DN_FQ', 'L_OUT_FQ', 'LS_VEG1', 'LS_FRUIT'
]
continuous_columns = ['HE_wt', 'HE_ht', 'HE_BMI']  # 연속형 변수

for col in low_missing_columns + continuous_columns:
    if col in data.columns:
        data[col] = pd.to_numeric(data[col], errors='coerce')
        data[col] = data[col].replace([9, 99, 999, 88, 888, 9.0, 99.0, 999.0, 88.0, 888.0], np.nan)
        if not data[col].isnull().all():
            if col in continuous_columns:
                median_value = data[col].median()
                data[col] = data[col].fillna(median_value)
                print(f"Median for {col}: {median_value}")
            else:
                mode_series = data[col].mode(dropna=True)
                if not mode_series.empty:
                    mode_value = mode_series[0]
                    data[col] = data[col].fillna(mode_value)
                    print(f"Mode for {col}: {mode_value}")
                else:
                    print(f"No valid mode for {col}, leaving as is.")
        else:
            print(f"Column {col} is all NaN, skipping.")

#생체 지표 결측치 처리
knn_columns = [
    'HE_sbp1', 'HE_dbp1', 'HE_sbp2', 'HE_dbp2', 'HE_wc', 'HE_glu', 'HE_HbA1c', 'HE_chol',
    'HE_HDL_st2', 'HE_TG', 'HE_LDL_drct', 'HE_HCHOL', 'HE_BUN', 'HE_crea', 'HE_Upro', 'HE_Ucrea',
    'HE_ast', 'HE_alt', 'HE_wt', 'HE_ht', 'HE_BMI'
]
knn_data = data[knn_columns].copy()
knn_data = knn_data.replace([888, 999, 888.0, 999.0], np.nan)

# 스케일링
scaler = StandardScaler()
knn_data_scaled = scaler.fit_transform(knn_data)
knn_data_scaled = pd.DataFrame(knn_data_scaled, columns=knn_columns, index=knn_data.index)

# KNN Imputation
imputer_knn = KNNImputer(n_neighbors=5, weights="distance")
knn_imputed = imputer_knn.fit_transform(knn_data_scaled)

# 역스케일링
knn_imputed = scaler.inverse_transform(knn_imputed)
knn_imputed_data = pd.DataFrame(knn_imputed, columns=knn_columns, index=knn_data.index)
data[knn_columns] = knn_imputed_data

# knn_scaler_filename = 'HE_DM_HbA1c_knn_scaler.pkl'
# knn_imputer_filename = 'HE_DM_HbA1c_knn_imputer.pkl'
# joblib.dump(knn_data_scaled, knn_scaler_filename)
# joblib.dump(imputer_knn, knn_imputer_filename)
# knn_scaler_filename = 'HE_obe_knn_scaler.pkl'
# knn_imputer_filename = 'HE_obe_knn_imputer.pkl'
# joblib.dump(knn_data_scaled, knn_scaler_filename)
# joblib.dump(imputer_knn, knn_imputer_filename)
# knn_scaler_filename = 'HE_HP_knn_scaler.pkl'
# knn_imputer_filename = 'HE_HP_knn_imputer.pkl'
# joblib.dump(knn_data_scaled, knn_scaler_filename)
# joblib.dump(imputer_knn, knn_imputer_filename)

print("결측치 처리 후 데이터 상태:")
print(data.isnull().sum())
print("\n타겟 변수 클래스 비율:")
for target in ['HE_DM_HbA1c', 'HE_obe', 'HE_HP']:
    if target in data.columns:
        print(f"\n{target}:\n{data[target].value_counts(normalize=True)}")
print("\n기본 통계량:")
print(data.describe())
data.to_csv('processed_dataset_real2.csv', index=False, encoding='utf-8')


In [ ]:
data=pd.read_csv("C:/건강검진/health_checkup/JDM/Real/df_clustering_filled.csv")